In [1]:
import numpy as np 
import pandas
import cv2
from PIL import Image

In [9]:
img = cv2.imread("hxh.jpg")
i = cv2.imread("mori.jpg",cv2.COLOR_BGR2RGB)
p = cv2.imread("b.jpg",cv2.COLOR_BGR2RGB)
cv2.imshow("img",i)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
img.shape

(1080, 1920, 3)

In [4]:
img.dtype

dtype('uint8')

In [5]:
img.size

6220800

roate = np.float32([[0,])

In [173]:
r,c,_ = p.shape
roate = np.float32([[-1,0,c],[0,1,0],[0,0,1]])
rota = cv2.warpPerspective(p,roate,(c,r))
cv2.imshow("g",rota)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
zeros= np.zeros(img.shape[:2],np.uint8)
b,g,r = cv2.split(img)
# Displaying Green channel- image
cv2.imshow("b",cv2.merge([b, zeros, zeros]))
cv2.imshow("r",cv2.merge([zeros, zeros,r]))
cv2.imshow("g",cv2.merge([zeros, g,zeros]))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
grey = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2.imshow("greyscale",grey)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
i = cv2.resize(i,(1920,1080))

In [8]:
# add = cv2.add(grey,i)
add = cv2.addWeighted(img,0.5,i,0.5,0)
cv2.imshow("add",add)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [41]:
f = cv2.subtract(img,i)
cv2.imshow("f",f)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#333333333333333

In [13]:
def gaussian(img,mean = 0,std = 1):
    noise = np.random.normal(mean,std,img.shape).astype(np.uint8)
    noisy = cv2.add(img,noise)
    return noisy

In [14]:
noisy_img = gaussian(img,0,1)
cv2.imshow("Original Image", img)
cv2.imshow("Noisy Image", noisy_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
def sap(img,ratio=0.2):
    noisy = img.copy()
    a,b,c = img.shape
    rate = int(a*b*ratio)

    for _ in range(rate):
        r,c = np.random.randint(0,a),np.random.randint(0,b)
        if np.random.rand() < 0.5:
            noisy[r,c] = [0,0,0]
        else:
            noisy[r,c] = [255,255,255]
    return noisy

In [28]:
salt = sap(img,0.2)
cv2.imshow("Original Image", img)
cv2.imshow("Noisy Image", salt)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [30]:
mask = np.float32([[1,1,1],[1,1,1],[1,1,1]])
mask = mask/np.sum(mask)

gaussian = cv2.filter2D(noisy_img,-1,mask)
cv2.imshow("Noisy Image", gaussian)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [31]:
median =cv2.medianBlur(salt,5)
compare = np.concatenate((salt,median),axis = 1)
cv2.imshow('After Median filtering', compare)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [32]:
m = np.float32([[1,1,1],[0,0,0],[-1,-1,-1]])
sobelx = cv2.filter2D(img,-1,m)
cv2.imshow('After Median filtering', sobelx)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [75]:
m = np.float32([[1,0,-1],[1,0,-1],[1,0,-1]])
sobely = cv2.filter2D(img,-1,m)
s = np.clip(sobelx+sobely,0,255)
cv2.imshow('After Median filtering', s)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [82]:
def convol(img,m):
    r,c,_ = img.shape
    a,b = m.shape
    output = np.zeros_like(img)
    for i in range(r-a+1):
        for j in range(c-b+1):
            grid = img[i:i+a,j:j+b]
            summ = np.sum(grid*m)
            output[i,j] = summ
    output = np.clip(output,0,255)
    cv2.imshow("cob",output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [83]:
convol(i,m)

In [95]:
x = cv2.Sobel(img,-1,1,0,5)
y = cv2.Sobel(img,-1,0,1,5)
ad = cv2.add(x,y)
cv2.imshow("cob",ad)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [101]:
lap = cv2.Laplacian(img,-1)
cv2.imshow("cob",lap)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [102]:
gblur = cv2.GaussianBlur(img,(3,3),0)
lap = cv2.Laplacian(gblur,-1,3)
cv2.imshow("cob",lap)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [100]:
low = cv2.GaussianBlur(img,(3,3),0)
high = cv2.GaussianBlur(img,(3,3),100)
dog = high - low
cv2.imshow("cob",dog)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [159]:
gray = cv2.cvtColor(p, cv2.COLOR_BGR2GRAY)
    
    # Convert to float32 (cornerHarris requires this)
gray = np.float32(gray)
    
    # Apply the Harris corner detection
corner = cv2.cornerHarris(gray, 2, 3, 0.05)
    
    # Dilate the corners to enhance them
corner = cv2.dilate(corner, None)

    
    # Highlight the corners in the copied image
p[corner > 0.01 * corner.max()] = [0, 0, 255]  # Mark corners with red color
    
    # Display the original image with corners highlighted
cv2.imshow("Corners detected", p)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [130]:
p.shape

(1226, 1590)

In [175]:
sift = cv2.SIFT_create()
g = np.copy(i)
kp = sift.detect(i,None)
pe = cv2.drawKeypoints(g,kp,i)
cv2.imshow("Corners detected", i)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
from skimage.feature import hog
from skimage import exposure

In [4]:
fd,hog_im = hog(
    p,
    orientations=9,
    pixels_per_cell=(8, 8),
    cells_per_block=(2, 2),
    visualize=True,
    channel_axis=-1,
)
new = exposure.rescale_intensity(hog_im,in_range=(0,10))
cv2.imshow("Corners detected", new)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
pixel_values = img.reshape((-1,3))
pixel_values = np.float32(pixel_values)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER,100,0.2)
k=50
_,labels,(centers) = cv2.kmeans(pixel_values,k,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
centers = np.uint8(centers)
labels = labels.flatten()
seg = centers[labels.flatten()]
seg = seg.reshape(img.shape)
cv2.imshow("Corners detected", seg)
cv2.waitKey(0)
cv2.destroyAllWindows()